<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Notebooks/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio == -qU
!pip install sentence-transformers -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.0 MB/s eta 0:

In [2]:
import transformers
import gradio as gr
from sentence_transformers import SentenceTransformer,util
import pickle
import pandas as pd
from huggingface_hub import hf_hub_download
import os

In [3]:
pd.options.mode.chained_assignment = None  # Turn off SettingWithCopyWarning

In [4]:
from huggingface_hub import login
login()

In [5]:
songs_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'spotify_transformed.csv'))
mappings = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='song_mappings.csv'))
verses_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'verses.csv'))
song_embeddings = pickle.load(open(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='embeddings.pkl'),'rb'))

In [6]:
mappings.head(5)

,Unnamed: 0,0
0,Got a bag full of feelings,2 Sugar (feat. Ayra Starr)
1,And a house full of regrets,2 Sugar (feat. Ayra Starr)
2,"This baggages can't fit in no plane, don't mea...",2 Sugar (feat. Ayra Starr)
3,I wanna know what's the reason why you moving ...,2 Sugar (feat. Ayra Starr)
4,As you move no dey cross my lane,2 Sugar (feat. Ayra Starr)


In [7]:
mappings.rename(columns={'Unnamed: 0':'verse','0':'song_name'},inplace=True)

In [8]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [10]:
verses_df.rename(columns={'0':'verse'},inplace=True)

In [69]:
songs_df.iloc[25:30].image

25    https://i.scdn.co/image/ab67616d0000b27385adb4...
26    https://i.scdn.co/image/ab67616d00001e0285adb4...
27    https://i.scdn.co/image/ab67616d0000485185adb4...
28    https://i.scdn.co/image/ab67616d0000b27351f2cf...
29    https://i.scdn.co/image/ab67616d00001e0251f2cf...
Name: image, dtype: object

In [47]:
prompt = 'i need igbo and shayo'
prompt_embed = embedder.encode(prompt)
hits = util.semantic_search(prompt_embed,song_embeddings,top_k=30)
hits = pd.DataFrame.from_dict(hits[0])
verses_match = verses_df.iloc[hits['corpus_id']]
songs_match = mappings[mappings['verse'].isin(verses_match['verse'].values)]
songs_match = songs_df[songs_df['song_name'].isin(songs_match['song_name'].values)]
songs_match = songs_match.sort_values('song_name')
songs_match = songs_match.drop_duplicates(subset='song_name')
songs_name = list(songs_match['song_name'][:9])
cover_art = list(songs_match['image'][:9])

In [48]:
assert len(songs_name) == len(cover_art)

In [ ]:
cover_art

In [63]:
def generate_playlist(prompt):
  prompt_embed = embedder.encode(prompt)
  hits = util.semantic_search(prompt_embed,song_embeddings,top_k=30)
  hits = pd.DataFrame.from_dict(hits[0])
  verses_match = verses_df.iloc[hits['corpus_id']]
  songs_match = mappings[mappings['verse'].isin(verses_match['verse'].values)]
  songs_match = songs_df[songs_df['song_name'].isin(songs_match['song_name'].values)]
  songs_match = songs_match.sort_values('song_name')
  songs_match = songs_match.drop_duplicates(subset='song_name')
  songs_name = list(songs_match['song_name'][:9])
  cover_art = list(songs_match['image'][:9])
  images = [gr.Image.update(value=art,visible=True) for art in cover_art]
  return (gr.Radio.update(label='songs',interactive=True,choices=songs_name),
          *images)


In [46]:
def set_example_prompt(examples):
  return gr.TextArea.update(value=examples[0])

In [65]:
demo = gr.Blocks()
with demo:
  gr.Markdown(
      '''
      # A Text based playlist Generator for Afrobeats

      '''
  )
  with gr.Row():
    with gr.Column():
      gr.Markdown(
          '''
          Enter a Prompt to generate a playlist based on that prompt
          '''
      )
      song_prompt = gr.TextArea(
          value='I need igbo and shayo',
          placeholder = "Enter a sentence that describes how you're feeling or what you want your playlist to be about"
      )
      example_prompts = gr.Dataset(
          components=[song_prompt],
          samples= [
              ['Lazy day'],
              ['I need money'],
              ['nobody gets me ']
          ]
      )
    with gr.Column():
      fetch_songs = gr.Button(value='generate your playlist',).style(full_width=True)

    with gr.Column():
        song_options = gr.Radio(label='songs',interactive=True,choices=None,type='value',visible=True)
    with gr.Column():
      with gr.Row():
        tile1 = gr.Image(value="/content/songs_cover.jpg", show_label=False, visible=True)
        tile2 = gr.Image(value="/content/songs_cover.jpg", show_label=False, visible=True)
        tile3 = gr.Image(value="/content/songs_cover.jpg", show_label=False, visible=True)
      with gr.Row():
        tile4 = gr.Image(value="/content/songs_cover.jpg", show_label=False,visible=True)
        tile5 = gr.Image(value="/content/songs_cover.jpg", show_label=False,visible=True)
        tile6 = gr.Image(value="/content/songs_cover.jpg", show_label=False,visible=True)
      with gr.Row():
        tile7 = gr.Image(value="/content/songs_cover.jpg",show_label=False,visible=True)
        tile8 = gr.Image(value='/content/songs_cover.jpg',show_label=False,visible=True)

      tiles = [tile1,tile2,tile3,tile4,tile5,tile6,tile7,tile8]


    fetch_songs.click(
        fn=generate_playlist,
        inputs=[song_prompt],
        outputs=[song_options,*tiles]
    )
    example_prompts.click(
        fn=set_example_prompt,
        inputs = [example_prompts],
        outputs = example_prompts.components
    )


  demo.launch(debug=True)

<ipython-input-65-4b6f90c910fb>:29: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  fetch_songs = gr.Button(value='generate your playlist',).style(full_width=True)
<ipython-input-65-4b6f90c910fb>:29: GradioDeprecationWarning: Use `scale` in place of full_width in the constructor. scale=1 will make the button expand, whereas 0 will not.
  fetch_songs = gr.Button(value='generate your playlist',).style(full_width=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1392, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1288, in postprocess_data
    self.validate_outputs(fn_index, predictions)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1263, in validate_outputs
    raise ValueError(
ValueError: An event handler (generate_playlist) didn't receive enough output values (needed: 9, received: 8).
Wanted outputs:
    [radio, image, image, image, image, image, image, image, image]
Received outputs:
    [{'choices': ['Culture', 'Door', 'Free Mind', 'Last Last', 'Logba Logba', 'Time N Affection', 'attention'], 'label': 'songs', 'info': None, 'show_label': None,

Keyboard interruption in main thread... closing server.
